In [10]:
import pandas as pd
import numpy as np

In [32]:
files = ['Data/Customer_review_data/Apex AD2600 Progressive-scan DVD player.txt',
         'Data/Customer_review_data/Canon G3.txt',
         'Data/Customer_review_data/Creative Labs Nomad Jukebox Zen Xtra 40GB.txt',
         'Data/Customer_review_data/Nikon coolpix 4300.txt',
         'Data/Customer_review_data/Nokia 6610.txt',
         'Data/CustomerReviews-3_domains/Computer.txt',
         'Data/CustomerReviews-3_domains/Router.txt',
         'Data/CustomerReviews-3_domains/Speaker.txt',
         'Data/Reviews-9-products/Canon PowerShot SD500.txt',
         'Data/Reviews-9-products/Canon S100.txt',
         'Data/Reviews-9-products/Diaper Champ.txt',
         'Data/Reviews-9-products/Hitachi router.txt',
         'Data/Reviews-9-products/ipod.txt',
         'Data/Reviews-9-products/Linksys Router.txt',
         'Data/Reviews-9-products/MicroMP3.txt',
         'Data/Reviews-9-products/Nokia 6600.txt',
         'Data/Reviews-9-products/norton.txt']

In [33]:
def read_file(file_path):
    refined_reviews = []
    
    with open(file_path, 'r') as file:
        text = file.read()
        reviews = text.strip().split('\n')

        if reviews[0] == '*****************************************************************************':
            reviews = reviews[11:]
        
        for review in reviews:
            parts = review.split('##')
            
            if len(parts) > 1:
                tags = parts[0].strip().split(',')
                content = parts[1].strip() 
            else:
                tags = []
                content = parts
                
            refined_reviews.append((tags, content))
    
        for rewiew in refined_reviews[0:20]:
            print(rewiew)
            print('\n')



for file in files:
    print('\n')
    print(file)
    print('\n')
    read_file(file)
    print('\n')
    print('\n')
    print('\n')



Data/Customer_review_data/Apex AD2600 Progressive-scan DVD player.txt


([], ['[t] troubleshooting ad-2500 and ad-2600 no picture scrolling b/w . '])


([''], 'repost from january 13 , 2004 with a better fit title .')


([''], 'does your apex dvd player only play dvd audio without video ?')


([''], 'or does it play audio and video but scrolling in black and white ?')


([''], 'before you try to return the player or waste hours calling apex tech support , or run the player over with your car , try these simple troubleshooting ideas first .')


([''], 'no picture :')


([''], 'hopefully you still have the remote control .')


([''], 'if you tossed it out the window , you need to fetch it .')


([''], 'using the remote control , press the i/p button located on the bottom right corner of the remote .')


([''], 'the i/p button switches the tv display between interlace and progressive .')


([''], 'if this doesnt bring back the picture , try pressing this button without playing a dvd .')